In [1]:
from pathlib import Path
import time
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import kampo
import openbabel
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from plip.structure.preparation import PDBComplex
from plip.exchange.report import BindingSiteReport
#from kampo.interactions import analyzer
#from kampo.converters import obabel, bp

In [2]:
base_work_dir = Path.cwd()
base_work_dir = base_work_dir.resolve()
work_dir = base_work_dir
pdbqt_files = sorted(list((work_dir / 'data').glob("*.pdbqt")))
for f in pdbqt_files:
    kampo.converters.obabel.pdbqt2sdf(f"{f}", str(work_dir / 'data' / f"{f.stem}.sdf"))

6 molecules converted


In [3]:
sdf_filepath = str(work_dir / 'data' / "ligand_0_0-0_out.sdf") # @param {type: "string"}
pdb_filepath = str(work_dir / 'data' /"5ndd_sligkv_lig9_frame_95_alignto_opm.pdb") # @param {type: "string"}
output_path = str(work_dir / 'data' /"complex.pdb") # @param {type: "string"}
kampo.converters.bp.getComplex(sdf_filepath, pdb_filepath, output_path)

Done! Complex saved to /Users/yuzotakagi/dev/drug/kampo/examples/data/complex.pdb


In [5]:
interactions_by_site = kampo.interactions.analyzer.retrieve_plip_interactions(output_path)
index_of_selected_site = 0
selected_site = list(interactions_by_site.keys())[index_of_selected_site]


In [7]:
kampo.interactions.analyzer.create_df_from_binding_site(interactions_by_site[selected_site], interaction_type="hbond")

,RESNR,RESTYPE,RESCHAIN,RESNR_LIG,RESTYPE_LIG,RESCHAIN_LIG,SIDECHAIN,DIST_H-A,DIST_D-A,DON_ANGLE,PROTISDON,DONORIDX,DONORTYPE,ACCEPTORIDX,ACCEPTORTYPE,LIGCOO,PROTCOO
0,326,TYR,1,1,UNL,Z,True,2.59,3.45,147.99,True,5919,O3,6465,N2,"(4.836, -1.373, 15.012)","(3.284, -4.409, 14.516)"
1,156,TYR,1,1,UNL,Z,True,3.48,3.91,109.17,True,1585,O3,6461,O2,"(3.901, 2.487, 15.662)","(1.735, -0.143, 13.746)"
2,227,HIS,1,1,UNL,Z,True,3.27,3.67,106.82,True,2741,Npl,6461,O2,"(3.901, 2.487, 15.662)","(0.789, 0.621, 16.236)"
3,227,HIS,1,1,UNL,Z,False,2.99,3.85,140.69,False,6463,Nam,2736,O2,"(3.641, 0.294, 16.195)","(2.713, 1.0, 19.869)"
